In [1]:
import numpy as np
from harfe import harfe
from harfe.utils import *
import csv


### Load data and build the training and validation set

In [2]:
xtr = csv.reader(open("propulsion_Xtrain.csv"), delimiter = ",")
xte = csv.reader(open("propulsion_Xval.csv"), delimiter = ",")
ytr = csv.reader(open("propulsion_Ytrain.csv"), delimiter = ",")
yte = csv.reader(open("propulsion_Yval.csv"), delimiter = ",")

X_train = np.array(list(xtr)).astype("float")
X_val = np.array(list(xte)).astype("float")
Y_train = (np.array(list(ytr)).astype("float")).flatten()
Y_val = (np.array(list(yte)).astype("float")).flatten()


### Define the relevant hyperparameters

In [3]:
num_data_train = X_train.shape[0]                # number of data points for training data set
num_data_val = X_val.shape[0]                    # number of data points for validation data set
num_data_tot = num_data_train + num_data_val     # total points
Nf = 15*num_data_train                           # number of features

d = X_train.shape[1]                             # input dimension for target function; 
q = 2                                            # order of the function
 
act = 'sin'                                      # activation function: sine, tanh, or relu
Dict_type = 'SRF'                                # Dictionary type of the form sin(Wx + b)

### Build the random feature matrix by sampling random weights and biases. 
### Use the HARFE algorithm to learn the coefficients and recover the function on validation set.

In [4]:
omega, bias = generate_omega_bias(rows = Nf, columns = d, weight = 1, par1 = -1, par2 = 1,
                                 distribution = 'uniform', bool_bias = True, sparsity = q)
    
A_train = feature_matrix(X_train, omega, bias, activation = act, dictType = Dict_type)
A_val = feature_matrix(X_val, omega, bias, activation = act, dictType = Dict_type)
    
scale_A = np.linalg.norm(A_train, axis = 0) 

A_train /= scale_A 
A_val /= scale_A
    
coeff, rel_error, iterations, _ = harfe(Y_train, A_train, N = Nf, s = 300, lam = 1e-10)


### Recover learnt function on validation set 

In [5]:
# recover validation points from learnt coefficients
y_val_rec = np.matmul(A_val, coeff)

# Calculate the mean squared error on validation set
mse = np.linalg.norm(y_val_rec.reshape(-1) - Y_val)**2/num_data_val
print('Mean square error:', mse, '\nIterations required:', iterations)

Mean square error: 6.908737473599776e-05 
Iterations required: 2
